<a href="https://colab.research.google.com/github/Mtakada30/hello-world/blob/main/20230704_getImage_dist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import matplotlib.pyplot as plt

Webから取ってくる。(その1)

In [ ]:
import requests
import os

def download_image(url, filepath):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filepath, 'wb') as f:
            f.write(response.content)

# 画像のURL
url = "https://www.tama.ac.jp/guide/teacher/img/000006tate.jpg"

# ファイルへの保存パス
filepath = "/content/photo.jpg"

# 関数を呼び出し、画像をダウンロード
download_image(url, filepath)

img = cv2.imread(filepath)

plt.imshow(img[:,:,::-1])

Webから取ってくる。(その2)

In [ ]:
!wget -O photo.jpg https://www.tama.ac.jp/guide/teacher/img/000006tate.jpg
photo = cv2.imread('photo.jpg')
plt.imshow(photo[:,:,::-1])

PCにあるファイルを使う

In [ ]:
from google.colab import files

uploaded_file = files.upload()
photo_path = next(iter(uploaded_file))
photo = cv2.imread(photo_path)

plt.imshow(photo[:,:,::-1])

Webカメラで取得する。

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  display(Image(filename))
except Exception as err:
  print(str(err))